In [64]:
import pandas as pd
import numpy as np
import os

file = os.path.join('Resources', 'purchase_data.json')

pur_data = pd.read_json(file)

pur_data.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [65]:
player_count = len(pur_data['SN'].unique())

players_df = pd.DataFrame([{'Total Players': player_count}])
players_df.set_index('Total Players', inplace = True)
players_df

""
Total Players
573


In [87]:
#pur_data['Item ID'].value_counts()
#unique_items = pd.DataFrame(pur_data['Item ID'].unique())
#len(unique_items)

no_dup_items = pur_data.drop_duplicates(['Item ID'], keep = 'last')
total_unique = len(no_dup_items)
sum_prices = no_dup_items['Price'].sum()
avg_price = sum_prices/total_unique
avg_price = round(avg_price, 2)

total_pur = pur_data['Price'].count()
total_rev = round(pur_data['Price'].sum(),2)
total_rev

# Number of Unique Items
# Average Purchase Price
# Total Number of Purchases
# Total Revenue

pur_analysis = pd.DataFrame([{
    
    "Number of Unique Items": total_unique,
    'Average Purchase Price': avg_price,
    'Total Purchases': total_pur,
    'Total Revenue': total_rev
}])

pur_analysis

,Average Purchase Price,Number of Unique Items,Total Purchases,Total Revenue
0,2.95,183,780,2286.33


In [111]:
# Gender Demographics

# Percentage and Count of Male Players
# Percentage and Count of Female Players
# Percentage and Count of Other / Non-Disclosed

no_dup_players = pur_data.drop_duplicates(['SN'], keep ='last')
#player_count from above
gender_counts = no_dup_players['Gender'].value_counts().reset_index()
gender_counts['% of Players'] = gender_counts['Gender']/player_count * 100
gender_counts.rename(columns = {'index': 'Gender', 'Gender': '# of Players'}, inplace = True)
gender_counts.set_index('Gender')
#gender_counts['% of Players'].sum()
gender_counts.style.format({"% of Players": "{:.1f}%"})

,Gender,# of Players,% of Players
0,Male,465,81.2%
1,Female,100,17.5%
2,Other / Non-Disclosed,8,1.4%


In [112]:
pur_data.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [169]:
# Purchasing Analysis (Gender)

# The below each broken by gender
# Purchase Count
# Average Purchase Price
# Total Purchase Value
# Normalized Totals

pur_count_by_gen = pd.DataFrame(pur_data.groupby('Gender')['Gender'].count())
total_pur_by_gen = pd.DataFrame(pur_data.groupby('Gender')['Price'].sum())
pur_analysis_gen = pd.merge(pur_count_by_gen, total_pur_by_gen, left_index = True, right_index = True)
pur_analysis_gen.rename(columns = {'Gender': '# of Purchases', 'Price':'Total Purchase Value'}, inplace=True)
pur_analysis_gen['Average Purchase Price'] = pur_analysis_gen['Total Purchase Value']/pur_analysis_gen['# of Purchases']
pur_analysis_gen = pur_analysis_gen.merge(gender_counts, left_index = True, right_on = 'Gender')
pur_analysis_gen['Normalized Totals'] = pur_analysis_gen['Total Purchase Value']/pur_analysis_gen['# of Players']
del pur_analysis_gen['% of Players']
del pur_analysis_gen['# of Players']
pur_analysis_gen.set_index('Gender', inplace=True)
pur_analysis_gen.style.format({'Total Purchase Value': '${:.2f}', 'Average Purchase Price': '${:.2f}', 'Normalized Totals': '${:.2f}'})

,# of Purchases,Total Purchase Value,Average Purchase Price,Normalized Totals
Gender,,,,
Female,136,$382.91,$2.82,$3.83
Male,633,$1867.68,$2.95,$4.02
Other / Non-Disclosed,11,$35.74,$3.25,$4.47
